In [1]:
from geopy.geocoders import Nominatim
import random
import requests
import numpy as np

API_ENDPOINT = "https://maps.googleapis.com/maps/api/staticmap"
STATIC_PARAMS = "maptype=satellite&format=jpg&zoom=20&size=512x512"
API_KEY = "read_from_credentials_file"


def get_images_from_city(city, num_per_city=40, folder="./images"):
    geolocator = Nominatim(user_agent="Portugal_rooftops")
    location = geolocator.geocode(f"{city}, Portugal")

    bb = [abs(float(coord)) for coord in location.raw['boundingbox']]
    mu_lat = (bb[1] + bb[0]) / 2
    mu_lng = (bb[3] + bb[2]) / 2
    sigma_lat = (bb[1] - bb[0]) / 36 # 3 sigma
    sigma_lng = (bb[2] - bb[3]) / 36 # 3 sigma
    lat = [str(p) for p in np.random.normal(mu_lat, sigma_lat, num_per_city)]
    lng = [f"-{str(p)}" for p in np.random.normal(mu_lng, sigma_lng, num_per_city)]
    points = [",".join(p) for p in list(zip(lat, lng))]

    for point in points:
        url = f"{API_ENDPOINT}?{STATIC_PARAMS}&center={point}&key={API_KEY}"
        img = requests.get(url)
        with open(f'{folder}/{city}_{point}.jpg', 'wb') as f:
            f.write(img.content)
        f.close()

In [2]:
portugal_cities = ["Aveiro", "Braga", "Coimbra", "Evora", "Guimaraes", "Lisbon", "Porto",
                   "Sintra", "Viseu", "Matosinhos"]

for city in portugal_cities:
    get_images_from_city(city)